<a href="https://colab.research.google.com/github/GilbertoRocha/Estudos/blob/master/DataScience/TensorFlow/AutoEncoder/base_credito_classificacao_estimator_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercicio extraido do curso "TensorFlow: Machine Learning e Deep Learning com Python", oferecido pelo Prof. Jones Granatyr. Link do curso: https://www.udemy.com/tensorflow-machine-learning-deep-learning-python/

Adicionado comentários para melhor entendimento.

In [2]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"gilbertorocha","key":"4d3618c473ea67622c0399535290b8ff"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

!chmod 600 /root/.kaggle/kaggle.json


!kaggle datasets download -d gilbertorocha/credit-data-for-tensorflow-autoencoder -p /content

!unzip \*.zip

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
  0% 0.00/57.7k [00:00<?, ?B/s]
100% 57.7k/57.7k [00:00<00:00, 49.8MB/s]
Archive:  credit-data-for-tensorflow-autoencoder.zip
  inflating: credit_data.csv         


In [3]:
# prepara os dados para a classificação, e para o encode
import pandas as pd
base = pd.read_csv('credit_data.csv')
base = base.drop('i#clientid', axis = 1)
base = base.dropna()
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
X = base.drop('c#default', axis = 1)
y = base['c#default']
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [0]:
# monta a estrutura da rede
# numero de atributos que serão "encodados"
neuronios_entrada = 3

# numero de atributos do "encode", ou seja, os atributos que vão representar as entradas
neuronios_oculta = 2

# numero de atributos de saida, precisa ser igual aos de entrada, pois o encode deve
# sempre representar a entrada fielmente, e os neuronios de saida permitem isso
# ficando na estrutura 3 -> 2 -> 3
neuronios_saida = neuronios_entrada


In [0]:
import tensorflow as tf

In [0]:
# criação dos placeholder para receberem uma carga de atributos
# tf.float32 = tipo da entrada, um float
# shape = formato da entrada
#       None = Sem tamanho definido do lote (sera definido mais tarde)
#       neuronios_entrada = quantidade de entrada
xph = tf.placeholder(tf.float32, shape = [None, neuronios_entrada])

In [0]:
# importa uma camada densa de rede neural, aonde todos os neuronios da camanda se conectam a todos os neuronios da proxima camada
from tensorflow.contrib.layers import fully_connected

# monta a camada oculta densa
#   inputs = recebe o place holder contendo as entradas
#   num_outputs = recebe a quantidade de saida dos neuronios (neste caso, a saida é o dado com encode)
#   activation_fn = funcao de ativacao, não é passada nenhuma, pois nao é interessante aplicar em um autoencoder
camada_oculta = fully_connected(inputs = xph, num_outputs = neuronios_oculta, activation_fn = None)

# monta a camada de saida da rede (entradas ja foram encodadas, e agora volta ao estado original)
#    inputs = recebe a camanda oculta, pois esta será a saida
#    num_outputs = recebe o numero dos neuronios da camanda de saida
camada_saida = fully_connected(inputs = camada_oculta, num_outputs = neuronios_saida)


In [8]:
# cria a funcao de erro
#    labels = são as respostas corretas, neste caso como e um autoencoder, as saidas devem ser identicas as entradas
erro = tf.losses.mean_squared_error(labels = xph, predictions = camada_saida)

# cria o otimizador
otimizador = tf.train.AdamOptimizer(0.01)

# cria a funcao de treinamento
treinamento = otimizador.minimize(erro)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoca in range(1000):
    # custo = recebe o valor da funcao otimizador
    # _ = recebe o valor do retorno da funcao treinamento, mas como ela não retorna nada, deve ser ignorado (por isso o underline)
    # feed_dict = alimenta a sessao
    custo, _ = sess.run([erro,treinamento], feed_dict = {xph: X})
    if epoca % 100 ==0:
      print('erro: ' + str(custo))
  # recupera agora a camada oculta, que seria o encode de 2 atributos
  x2d_encode = sess.run(camada_oculta, feed_dict = {xph: X})
  # recupera a camada de saida, com os 3 atributos originais (deveriam ser)
  x3d_decode = sess.run(camada_saida, feed_dict = {xph: X})

erro: 1.5343426
erro: 0.66898406
erro: 0.5721266
erro: 0.5632155
erro: 0.56071556
erro: 0.56038076
erro: 0.5602559
erro: 0.560237
erro: 0.5602369
erro: 0.5602368


In [10]:
x2d_encode.shape

(1997, 2)

In [11]:
x3d_decode.shape

(1997, 3)

In [12]:
# desescalona, para visualizar
X2 = scaler_x.inverse_transform(X)
X2

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [13]:
# desescalona os dados ja decodificados, para ver se o encode funcionou
x3d_decode2 = scaler_x.inverse_transform(x3d_decode)
x3d_decode2

array([[6.2037574e+04, 5.8928295e+01, 9.2291807e+03],
       [4.7633383e+04, 4.8534340e+01, 4.4454878e+03],
       [5.8417953e+04, 6.3183090e+01, 7.6981641e+03],
       ...,
       [5.0208191e+04, 4.0807560e+01, 4.4454878e+03],
       [4.5333863e+04, 6.3698540e+01, 4.4454878e+03],
       [6.2191008e+04, 5.5931473e+01, 9.2512363e+03]], dtype=float32)

In [14]:
# para medir a precisao, pode-se usar o mean absolute error
from sklearn.metrics import mean_absolute_error
mae_income = mean_absolute_error(X2[:,0], x3d_decode2[:,0])
mae_income

9397.151988189991

In [15]:
mae_age = mean_absolute_error(X2[:,1], x3d_decode2[:,1])
mae_age

5.956022837813002

In [16]:
mae_loan = mean_absolute_error(X2[:,2], x3d_decode2[:,2])
mae_loan

1802.9975616917957

In [0]:
# cria um dataframe para testar com o algoritimo de classificação

X_encode = pd.DataFrame({'atributo1': x2d_encode[:, 0], 'atributo2': x2d_encode[:, 1], 'classe': y})

In [19]:
X_encode.head()

,atributo1,atributo2,classe
0,0.945148,3.281457,0
1,0.811342,-0.166611,0
2,1.682261,2.536032,0
3,0.343701,0.353172,0
4,-3.744917,3.088584,1


In [23]:
# inicia a classificação usando a base com encode
colunas = [tf.feature_column.numeric_column(key = column) for column in X_encode.columns]

from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_encode, y, test_size = 0.3, random_state = 0)

funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento,
                                                        y = y_treinamento,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)

classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)

funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = y_teste,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)

metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpug32xjn0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3002563cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create 

In [24]:
metricas_teste

{'accuracy': 1.0,
 'accuracy_baseline': 0.8635,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.0026806805,
 'global_step': 1000,
 'label/mean': 0.1365,
 'loss': 0.021445444,
 'precision': 1.0,
 'prediction/mean': 0.13787064,
 'recall': 1.0}